<a href="https://colab.research.google.com/github/zhaocaiQ/Git-Tutorial/blob/master/keyword_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터추출 쿼리

##pymysql설치

In [1]:
pip install pymysql

     |████████████████████████████████| 43 kB 1.4 MB/s 


In [12]:
import pymysql
#데이터베이스 연동
conn = pymysql.connect(host='', port=3306, user='', password='', db='', charset='utf8')
cursor = conn.cursor()

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

In [ ]:
print(date)

2022-02-27


''' 레이블은 레이블대로, 데이터는 데이터대로 리스트 파일에 담아야 차트에 바로 적용 가능 '''

##daily_top25

In [13]:
#DB keyword테이블에서 오늘 기준 키워드 카운트 순 상위 25개의 keyword 추출
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = curdate() ORDER BY weight DESC LIMIT 25;")
result = dict(cursor.fetchall())
#오늘에 대한 정보 없을 시 어제날짜로 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  result = dict(cursor.fetchall())

#각 리스트에 keyword와 weight 담기
labels = list(result.keys())
data = list(result.values())
print(labels)
print(data)

['우크라', '러시아', '확진', '사망', '대선', '푸틴', '차별', '신규', '토론', '마지막', '바이든', '개학', '폭격', '일교차', '투표', '포격', '러시아군', '키예프', '최다', '연설', '전쟁', '이우', '민간', '세계', '여론']
[31, 26, 21, 17, 15, 14, 12, 12, 11, 11, 11, 10, 10, 10, 9, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7]


##weekly_top25

In [14]:
#DB keyword테이블에서 이번주 기준 keyword별 언급량 합계 상위 25개의 keyword 추출
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = dict(cursor.fetchall())
#이번주에 대한 정보가 없을시 지난주에 대한 정보 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
  result = dict(cursor.fetchall())

#각 리스트에 keyword와 weight 담기
labels = list(result.keys())
data = list(result.values())
print(labels)
print(data)

['우크라', '러시아', '확진', '산불', '대선', '푸틴', '전쟁', '제재', '이재명', '단일', '방역', '협상', '민간', '윤석열', '정부', '일교차', '키예프', '국제', '사망', '포격', '진화', '바이든', '폭격', '합천', '결렬']
[97, 83, 41, 34, 31, 30, 27, 26, 25, 24, 23, 23, 21, 20, 19, 18, 18, 18, 17, 17, 16, 16, 15, 15, 14]


##monthly_top25

In [15]:
#DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 25개의 keyword 추출
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format(curdate(), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = dict(cursor.fetchall())
#이번달에 대한 정보가 없을시 지난달에 대한 정보 출력
if len(result) == 0:
  cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
  result = dict(cursor.fetchall())

#각 리스트에 keyword와 weight 담기
labels = list(result.keys())
data = list(result.values())
print(labels)
print(data)

['우크라', '러시아', '확진', '푸틴', '산불', '대선', '방역', '민간', '이재명', '정부', '전쟁', '제재', '포격', '사망', '진화', '바이든', '폭격', '합천', '패스', '중단', '신규', '차별', '국제', '고령', '마지막']
[62, 55, 33, 30, 26, 23, 23, 21, 19, 19, 18, 18, 17, 17, 16, 16, 15, 15, 14, 14, 12, 12, 11, 11, 11]


##daily_chart

In [16]:
#DB keyword테이블에서 어제, 오늘 키워드 카운트 순 상위 25개의 keyword와 weight추출
#오늘 top1~25출력
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = curdate() ORDER BY weight DESC LIMIT 25;")
today = dict(cursor.fetchall())
if len(today) != 0:
  #전날 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  yesterday = dict(cursor.fetchall())
else:
  #오늘 정보가 없을 시 어제,그저께 정보로 데이터 출력
  #어제 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
  today = dict(cursor.fetchall()) 
  #그저께 top1~25출력
  cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -2 day) ORDER BY weight DESC LIMIT 25;")
  yesterday = dict(cursor.fetchall())

daily_chart = []

#어제 비교, 오늘 키워드의 등락표시
for i in today.keys():
    if i in yesterday.keys():
      if today[i] > yesterday[i]:
          daily_chart.append([i, today[i], "up"])
      elif today[i] == yesterday[i]:
          daily_chart.append([i, today[i], "same"])
      else:
          daily_chart.append([i, today[i], "down"])
    else:
      daily_chart.append([i, today[i], "new"])

dic_data = {
    'data': daily_chart
}
print(dic_data)

{'data': [['우크라', 31, 'same'], ['러시아', 26, 'down'], ['확진', 21, 'up'], ['사망', 17, 'new'], ['대선', 15, 'up'], ['푸틴', 14, 'down'], ['차별', 12, 'new'], ['신규', 12, 'new'], ['토론', 11, 'new'], ['마지막', 11, 'new'], ['바이든', 11, 'new'], ['개학', 10, 'new'], ['폭격', 10, 'new'], ['일교차', 10, 'new'], ['투표', 9, 'new'], ['포격', 8, 'down'], ['러시아군', 8, 'new'], ['키예프', 8, 'new'], ['최다', 8, 'new'], ['연설', 8, 'new'], ['전쟁', 8, 'down'], ['이우', 8, 'new'], ['민간', 7, 'down'], ['세계', 7, 'new'], ['여론', 7, 'new']]}


##weekly_chart

In [17]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
#이번주(월~일) 날짜 추출
cursor.execute("WITH RECURSIVE day AS (SELECT ADDDATE(curdate(), - WEEKDAY(curdate())) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < ADDDATE(curdate(), - WEEKDAY(curdate())+ 6)) SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
day = cursor.fetchall()

if len(keywords) == 0:
  #이번주 데이터가 없을 시 지난주 데이터 추출
  #DB keyword테이블에서 지난주 keyword별 언급량 합계 상위 5개의 keyword 추출
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  #지난주(월~일) 날짜 추출
  cursor.execute("WITH RECURSIVE day AS (SELECT DATE(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate())) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < DATE(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())))) SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
  day = cursor.fetchall()

weekly_chart = {}
#DB keyword테이블에서 추출한 keywords의 일별 weight 추출
for i in range(5):
  weight = [0] * 7
  no = 0
  for d in day:
    if d[0] == date:
      break
    else:
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) = '{d[0]}' AND keyword = '{keywords[i][0]}'")
      daily_weight = list(cursor.fetchall())
      if daily_weight != []:
        weight[no] = daily_weight[0][0]
      no += 1
  weekly_chart[keywords[i][0]] = weight
print(weekly_chart)

{'우크라': [35, 31, 31, 0, 0, 0, 0], '러시아': [28, 29, 26, 0, 0, 0, 0], '확진': [8, 12, 21, 0, 0, 0, 0], '산불': [8, 26, 0, 0, 0, 0, 0], '대선': [8, 8, 15, 0, 0, 0, 0]}


##monthly_chart

In [18]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
#이번달(1일~말일) 날짜 추출
cursor.execute("WITH RECURSIVE day AS (SELECT DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < LAST_DAY(NOW()))SELECT date_format(DAY, '%Y-%m-%d') FROM day")
day = cursor.fetchall()

if len(keywords) == 0:
  #이번주 데이터가 없을 시 지난달 데이터 추출
  #DB keyword테이블에서 지난달 keyword별 언급량 합계 상위 5개의 keyword 추출
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  #지난달(1일~말일) 날짜 추출
  cursor.execute("WITH RECURSIVE day AS (SELECT DATE_SUB((curdate() - INTERVAL 1 MONTH), INTERVAL (DAY(curdate())-1) DAY) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < LAST_DAY((curdate() - INTERVAL 1 MONTH)))SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
  day = cursor.fetchall()

monthly_chart = {}

#DB keyword테이블에서 추출한 keywords의 일별 weight 추출
for i in range(5):
  weight = [0] * int(day[-1][0][-2:])
  for d in day:
    if d[0] == date:
      break
    else:
      #일별 추출한 keywords의 weight 추출
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) = '{d[0]}' AND keyword = '{keywords[i][0]}'")
      daily_weight = list(cursor.fetchall())
      if daily_weight != []:
        weight[int(d[0][-2:])-1] = daily_weight[0][0]
  monthly_chart[keywords[i][0]] = weight
print(weekly_chart)

{'우크라': [35, 31, 31, 0, 0, 0, 0], '러시아': [28, 29, 26, 0, 0, 0, 0], '확진': [8, 12, 21, 0, 0, 0, 0], '산불': [8, 26, 0, 0, 0, 0, 0], '대선': [8, 8, 15, 0, 0, 0, 0]}


''' 레이블은 레이블대로, 데이터는 데이터대로 리스트 파일에 담아야 차트에 바로 적용 가능 '''

##daily_twitter

In [19]:
#DB keyword테이블에서 오늘 기준 키워드 카운트 순 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB twitter테이블에서 추출한 keywords의 키워드, 리트윗 수, 좋아요 수, 해시태그 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) = date(now()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #오늘 정보 없을 시 어제 정보출력
  cursor.execute("SELECT keyword FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) = date_add(curdate(), interval -1 day) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

daily_twitter = {}
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            daily_twitter[word[0]] = tweet[1:]
    no += 1

labels = []
data = []

for i in range(len(daily_twitter.keys())):
    key = list(daily_twitter.keys())[i]
    value = list(daily_twitter.values())[i]
    value = list(value)
    value.insert(0, key)
    labels.append(i + 1)
    data.append(value)

print(labels)
print(data)

[1, 2, 3, 4, 5]
[['우크라', 183358, 29, 'KBS뉴스, KBS,윤석열, Ukraine, StopWar, StopPutin'], ['러시아', 267460, 150, 'SBS뉴스,더배트맨, TheBatman, 러시아'], ['확진', 187896, 1935, 'BTSARMY, BestFanArmy, iHeartAwards,SBS뉴스,전남도청, 재난문자,코로나19'], ['사망', 55784, 87, '산업재해, 인천, 경기일보,섹트, 디엠, 맘눌뎀, 직장씨발,코로나19'], ['대선', 67861, 4, '김동연, 정치개혁, 윤석열, 대선, 이재명,이재명_헬게이트, 윤석열에_투표하고_이재명을_응징하자']]


##weekly_twitter

In [20]:
#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB twitter테이블에서 추출한 keywords의 키워드, 리트윗 수, 좋아요 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #이번주 정보 없을 시 지난주 정보출력
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%y-%m-%d') AND SUBSTR(keyword_id, 1, 8) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%y-%m-%d') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

#twitter테이블에서 추출한 정보 keyword별로 합치기
weekly_twitter = {}
for i in keywords:
  weekly_twitter[i[0]] = [0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            weekly_twitter[word[0]][0] += tweet[1]
            weekly_twitter[word[0]][1] += tweet[2]
            if tweet[3] != None: #예외처리: 해시태그가 NULL일 경우 오류발생
              weekly_twitter[word[0]][2] += tweet[3]
    no += 1
print(weekly_twitter)

{'우크라': [234539, 129, 'KBS뉴스, KBS,윤석열, Ukraine, StopWar, StopPutinSBS뉴스,오늘의신문_스크랩SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담'], '러시아': [730091, 338, 'SBS뉴스,더배트맨, TheBatman, 러시아레이시즘_퍼레이드,삼일절, 독립운동, 대한민국만세PrayForUkraine,SBS뉴스'], '확진': [187896, 1935, 'BTSARMY, BestFanArmy, iHeartAwards,SBS뉴스,전남도청, 재난문자,코로나19'], '산불': [95147, 37, '2022산불상황,mbc,SBS뉴스,조건만남'], '대선': [67861, 4, '김동연, 정치개혁, 윤석열, 대선, 이재명,이재명_헬게이트, 윤석열에_투표하고_이재명을_응징하자']}


##monthly_twitter

In [22]:
# #DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB twitter테이블에서 추출한 keywords의 키워드, 리트윗 수, 좋아요 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 8) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND SUBSTR(keyword_id, 1, 8) <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()
else:
  #이번달 정보 없을 시 지난달 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE SUBSTR(keyword_id, 1, 5) = date_format((curdate() - INTERVAL 1 MONTH), '%y-%m') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  twitter = cursor.fetchall()

#twitter테이블에서 추출한 정보 keyword별로 합치기
monthly_twitter = {}
for i in keywords:
  monthly_twitter[i[0]] = [0, 0, '']

no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for tweet in twitter:
        if word[0] in tweet[0]:
            monthly_twitter[word[0]][0] += tweet[1]
            monthly_twitter[word[0]][1] += tweet[2]
            if tweet[3] != None:  #예외처리: 해시태그가 NULL일 경우 오류발생
              monthly_twitter[word[0]][2] += tweet[3]
    no += 1
print(monthly_twitter)

{'우크라': [191009, 86, 'KBS뉴스, KBS,윤석열, Ukraine, StopWar, StopPutinSBS뉴스,오늘의신문_스크랩'], '러시아': [472167, 183, 'SBS뉴스,더배트맨, TheBatman, 러시아레이시즘_퍼레이드,삼일절, 독립운동, 대한민국만세'], '확진': [187896, 1935, 'BTSARMY, BestFanArmy, iHeartAwards,SBS뉴스,전남도청, 재난문자,코로나19'], '푸틴': [936612, 139, 'Небензя, России, ДНР, ЛНР, Донбасса, Украины,코리안싸이코, 푸틴'], '산불': [95147, 37, '2022산불상황,mbc,SBS뉴스,조건만남']}


##daily_youtube

In [23]:
#DB keyword테이블에서 오늘 기준 키워드 카운트 순 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB youtube테이블에서 추출한 keywords의 키워드, 조회수, 좋아요 수, 댓글 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) = curdate() GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #오늘 정보 없을 시 어제 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 5;")
  keywords = list(cursor.fetchall())
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) = date_add(curdate(), interval -1 day) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()

daily_youtube = {}
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            daily_youtube[word[0]] = data[1:]
    no += 1
print(daily_youtube)

{'우크라': (14154355, 128226, 36570, '러시아, 우크라이나, 전쟁,format:기타, siries:SBS NEWS, siries:SBS 뉴스, siries:에스비에스 뉴스, station:SBS, station:SEOUL BROADCASTING SYSTEM, station:서울 방송, station:에스비에스, type:KOREA NEWS, type:Korean News, type:News, type:News Network, type:뉴스, type:대한민국 뉴스, type:방송, yt:cc=on, 러시아, 발견 즉시 덮어라, 백신, 어린이, 우크라, 접종, 화이자,source:영상, type:디지털, genre:정치, format:기타, series:자막뉴스, 이재명, 윤석열, 우크라이나, 러시아, 침공,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 미국, 러시아, 우크라이나, 젤렌스키,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 머스크, 일론머스크, 테슬라, 우크라이나, 전쟁,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 러시아, 전쟁, 포로, 러시아군, 군인, 푸틴, '), '러시아': (40165579, 279599, 53143, '군병원, 러 공수부대, 방송국,Article, NATO, Russia, U.S., Ukraine, War, allies, attack, civilian, forces, invasion, involvement, missiles, p_cmsid=2494279, p_vid=news-83186264, targets, troops,러시아군, 진격, 포위전,러시아, 우크라이나, 전쟁,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 전쟁, 러시아, Kyiv, Uk

##weekly_youtube

In [24]:
#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB youtube테이블에서 추출한 keywords의 키워드, 조회수, 좋아요 수, 댓글 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND SUBSTR(keyword_id, 1, 8) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #이번주 정보 없을 시 지난주 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall())  
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE SUBSTR(keyword_id, 1, 8) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%y-%m-%d') AND SUBSTR(keyword_id, 1, 8) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%y-%m-%d') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()

#youtube테이블에서 추출한 정보 keyword별로 합치기
weekly_youtube = {}
for i in keywords:
  weekly_youtube[i[0]] = [0, 0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            weekly_youtube[word[0]][0] += data[1]
            weekly_youtube[word[0]][1] += data[2]
            weekly_youtube[word[0]][2] += data[3]
            if data[4] != None:  #예외처리: 해시태그가 NULL일 경우 오류발생
              weekly_youtube[word[0]][3] += data[4]
    no += 1
print(weekly_youtube)

{'우크라': [35606865, 325298, 106173, '러시아, 우크라이나, 전쟁,format:기타, siries:SBS NEWS, siries:SBS 뉴스, siries:에스비에스 뉴스, station:SBS, station:SEOUL BROADCASTING SYSTEM, station:서울 방송, station:에스비에스, type:KOREA NEWS, type:Korean News, type:News, type:News Network, type:뉴스, type:대한민국 뉴스, type:방송, yt:cc=on, 러시아, 발견 즉시 덮어라, 백신, 어린이, 우크라, 접종, 화이자,source:영상, type:디지털, genre:정치, format:기타, series:자막뉴스, 이재명, 윤석열, 우크라이나, 러시아, 침공,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 미국, 러시아, 우크라이나, 젤렌스키,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 머스크, 일론머스크, 테슬라, 우크라이나, 전쟁,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 러시아, 전쟁, 포로, 러시아군, 군인, 푸틴, source:영상, type:방송, genre:국제, format:기타, person:한연희,format:기타, siries:SBS NEWS, siries:SBS 뉴스, siries:에스비에스 뉴스, station:SBS, station:SEOUL BROADCASTING SYSTEM, station:서울 방송, station:에스비에스, type:KOREA NEWS, type:Korean News, type:News, type:News Network, type:뉴스, type:대한민국 뉴스, type:방송, yt:cc=on, 대통령, 러시아, 바실렌코, 우크라, 의원,러시아, 우크라이나, 푸틴,MBN

##monthly_youtube

In [25]:
#DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB youtube테이블에서 추출한 keywords의 키워드, 조회수, 좋아요 수, 댓글 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube  WHERE SUBSTR(keyword_id, 1, 8) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND SUBSTR(keyword_id, 1, 8) <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #이번달 정보 없을 시 지난주 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall()) 
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube  WHERE SUBSTR(keyword_id, 1, 5) = date_format((curdate() - INTERVAL 1 MONTH), '%y-%m') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall() 

#youtube테이블에서 추출한 정보 keyword별로 합치기
monthly_youtube = {}
for i in keywords:
  monthly_youtube[i[0]] = [0, 0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            monthly_youtube[word[0]][0] += data[1]
            monthly_youtube[word[0]][1] += data[2]
            monthly_youtube[word[0]][2] += data[3]
            if data[4] != None:  #예외처리: 해시태그가 NULL일 경우 오류발생
              monthly_youtube[word[0]][3] += data[4]
    no += 1
print(monthly_youtube)

{'우크라': [27357404, 253096, 73915, '러시아, 우크라이나, 전쟁,format:기타, siries:SBS NEWS, siries:SBS 뉴스, siries:에스비에스 뉴스, station:SBS, station:SEOUL BROADCASTING SYSTEM, station:서울 방송, station:에스비에스, type:KOREA NEWS, type:Korean News, type:News, type:News Network, type:뉴스, type:대한민국 뉴스, type:방송, yt:cc=on, 러시아, 발견 즉시 덮어라, 백신, 어린이, 우크라, 접종, 화이자,source:영상, type:디지털, genre:정치, format:기타, series:자막뉴스, 이재명, 윤석열, 우크라이나, 러시아, 침공,MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 미국, 러시아, 우크라이나, 젤렌스키,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 머스크, 일론머스크, 테슬라, 우크라이나, 전쟁,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 러시아, 전쟁, 포로, 러시아군, 군인, 푸틴, source:영상, type:방송, genre:국제, format:기타, person:한연희,format:기타, siries:SBS NEWS, siries:SBS 뉴스, siries:에스비에스 뉴스, station:SBS, station:SEOUL BROADCASTING SYSTEM, station:서울 방송, station:에스비에스, type:KOREA NEWS, type:Korean News, type:News, type:News Network, type:뉴스, type:대한민국 뉴스, type:방송, yt:cc=on, 대통령, 러시아, 바실렌코, 우크라, 의원,러시아, 우크라이나, 푸틴,MBN,